In [1]:
import os.path

import pandas
import sourmash
from sourmash import sourmash_args

In [2]:
toplevel = '/home/zyzhao/assemloss/mock/mixD/'
grist_dir = toplevel + 'grist/outputs'
assembly_dir = toplevel + 'assembly/'
atta_dir = toplevel + 'atta/'

for path in (toplevel, grist_dir, assembly_dir, atta_dir):
    assert os.path.isdir(path), f'{path} is not a directory?'

In [9]:
class MetagenomeInfo:
    def __init__(self, metag_acc, *, ksize=31):
        self.metag_acc = metag_acc
        self.ksize = ksize

        self.metag_sig_path = os.path.join(grist_dir, "sigs",
                                           f"{self.metag_acc}.trim.sig.zip")
        self.metag_sig = sourmash_args.load_one_signature(self.metag_sig_path, ksize=self.ksize)

    def calc_assembly_stuff(self, assembly_dir):
        sigfile = os.path.join(assembly_dir, f"{self.metag_acc}.megahit.fa.gz.sig")
        assert os.path.exists(sigfile), sigfile

        self.assembly_sig = sourmash_args.load_one_signature(sigfile, ksize=self.ksize)

        # percent of flat k-mers accounted for by assembly
        print(f"assembly/unweighted: {self.metag_sig.contained_by(self.assembly_sig)*100:.1f}%")

        # abundance weighted version:
        assembly_mh = self.assembly_sig.minhash.flatten()
        metag_mh = self.metag_sig.minhash
        intersect = assembly_mh.intersection(metag_mh.flatten()).inflate(metag_mh)

        # now sum:
        total_weighted_sum = metag_mh.sum_abundances
        intersect_weighted_sum = intersect.sum_abundances
        print(f"assembly/weighted: {intersect_weighted_sum / total_weighted_sum * 100:.1f}%")

        sig_mapped = os.path.join(atta_dir, f'{self.metag_acc}.x.ma.fq.gz.sig')
        ma_sig = sourmash_args.load_one_signature(sigfile, ksize=self.ksize)
        print(f"% k-mers in reads mapped to assembly: {self.metag_sig.contained_by(ma_sig)*100:.1f}%")

    def calc_ref_based_kmer_stuff(self, grist_dir):
        gather_csv = os.path.join(grist_dir, "gather", f"{self.metag_acc}.gather.csv.gz")
        df = pandas.read_csv(gather_csv)
        row = df.tail(1).squeeze()
        sum_weighted_found = row['sum_weighted_found'] 
        total_weighted_hashes = row['total_weighted_hashes']
        # oops, this is the same as: print(df['f_unique_weighted'].sum())
        print(f"total ref k-mers found (abund): {sum_weighted_found / total_weighted_hashes * 100:.1f}")
        print(f"total ref k-mers found (flat): {df['f_unique_to_query'].sum() * 100:.1f}")

    def calc_mapping_stuff(self, grist_dir):
        leftover_csv = os.path.join(grist_dir, 'leftover',
                                    f"{self.metag_acc}.summary.csv")
        df = pandas.read_csv(leftover_csv)
        total_mapped_reads = df['n_mapped_reads'].sum()
        print(f"total mapped reads: {total_mapped_reads}")
        
        
metag_acc = "SRR24480607"
info = MetagenomeInfo(metag_acc)

info.calc_assembly_stuff(assembly_dir)
info.calc_ref_based_kmer_stuff(grist_dir)
info.calc_mapping_stuff(grist_dir)

assembly/unweighted: 52.6%
assembly/weighted: 95.3%
% k-mers in reads mapped to assembly: 52.6%
total ref k-mers found (abund): 96.7
total ref k-mers found (flat): 62.9
total mapped reads: 6526549
